In [32]:
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.svm import LinearSVC
import numpy as np
from linear_models import LinearOneVsAllClassifier
import os
from helper import generate_exp_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [12]:
mnist_train_images = np.copy(mnist.train.images)
mnist_train_labels = np.argmax(mnist.train.labels, axis=1)
mnist_test_images = mnist.test.images
mnist_test_labels = np.argmax(mnist.test.labels, axis=1)

In [10]:
num_models = 5
mnist_num_dim = 28 * 28
linear_models = []
zeroed_features_list = []
for i in xrange(num_models):
    sparse_mnist_images = np.copy(mnist_images)
    zeroed_features = np.random.randint(0, mnist_num_dim, int(.75 * mnist_num_dim))
    zeroed_features_list.append(zeroed_features)
    sparse_mnist_images[:, zeroed_features] = 0.0
    model = LinearSVC(loss='hinge')
    model.fit(sparse_mnist_images, mnist_labels)
    model = LinearOneVsAllClassifier(10, model.coef_, model.intercept_)
    linear_models.append(model)

In [13]:
[model.evaluate(mnist_test_images, mnist_test_labels) for model in linear_models]

[0.9152, 0.9138, 0.917, 0.9156, 0.9147]

In [20]:
# check that the models have no nonzero weights in any of the zeroed out dimensions
[sum(linear_models[i].weights[:,zeroed_features_list[i]].reshape(-1,)) for i in xrange(len(linear_models))]

[0.0, 0.0, 0.0, 0.0, 0.0]

In [22]:
os.mkdir('linear_models')

In [23]:
for i, model in enumerate(linear_models):
    np.save('linear_models/w_{}.npy'.format(i), model.weights)
    np.save('linear_models/b_{}.npy'.format(i), model.bias)

In [26]:
X_exp, Y_exp = generate_exp_data(1000, mnist_test_images, mnist_test_labels, linear_models)

In [27]:
[model.evaluate(X_exp, Y_exp) for model in linear_models]

[1.0, 1.0, 1.0, 1.0, 1.0]

In [28]:
os.mkdir('linear_experiments_data')

In [29]:
np.save('linear_experiments_data/X_exp_multi.npy', X_exp)
np.save('linear_experiments_data/Y_exp_multi.npy', Y_exp)